# Installations

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install ogb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.8/78.8 kB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for littleutils: filename=littleutils-0.2.2-py3-none-any.whl size=7029 sha256=d877d37ac9486f3687c00bff5b2b3a3e4f047dcd50eec767bb7dbdb2a7b7cfea
  Stored in directory: /root/.cache/pip/wheels/3d/fe/b0/27a9892da57472e538c7452a721a9cf463cc03cf7379889266
Successfully built littleutils


In [ ]:
!pip install torch_geometric

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 661.6/661.6 kB 12.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for torch_geometric: filename=torch_geometric-2.3.1-py3-none-any.whl size=910459 sha256=532e10b9d8787e2b10351c5153d43f4c6ce429aed56ebae48448e3c7d0f47bc3
  Stored in directory: /root/.cache/pip/wheels/ac/dc/30/e2874821ff308ee67dcd7a66dbde912411e19e35a1addda028
Successfully built torch_geometric


In [ ]:
!pip install rdkit

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached rdkit-2023.3.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (29.7 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 25.7 MB/s eta 0:00:00


In [ ]:
!pip install -q condacolab
import condacolab
condacolab.install()

✨🍰✨ Everything looks OK!


In [ ]:
!conda install -c conda-forge torch-scatter

Solving environment: \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ done


==> WARNING: A newer version of conda exists. <==
  current version: 23.3.1
  latest version: 23.5.0

Please update conda by running

    $ conda update -n base -c conda-forge conda

Or to minimize the number of packages updated during conda update use

     conda install conda=23.5.0



# All requested packages already installed.



In [ ]:
import torch
from torch_geometric.data import DataLoader
from torch_geometric.utils import to_dense_batch
from ogb.graphproppred import PygGraphPropPredDataset
import pandas as pd

# Load the ogbg-molhiv dataset
csv_path = '/content/drive/My Drive/Colab Notebooks/Molecule Classification/hiv.csv'
df = pd.read_csv(csv_path)
df.head(5)
df.drop('mol_id', axis='columns', inplace=True)
df.head(5)




,HIV_active,smiles
0,0,CCC1=[O+][Cu-3]2([O+]=C(CC)C1)[O+]=C(CC)CC(CC)...
1,0,C(=Cc1ccccc1)C1=[O+][Cu-3]2([O+]=C(C=Cc3ccccc3...
2,0,CC(=O)N1c2ccccc2Sc2c1ccc1ccccc21
3,0,Nc1ccc(C=Cc2ccc(N)cc2S(=O)(=O)O)c(S(=O)(=O)O)c1
4,0,O=S(=O)(O)CCS(=O)(=O)O


# Convert the SMILES strings to graphs

In [ ]:
from ogb.utils.features import (allowable_features, atom_to_feature_vector,
 bond_to_feature_vector, atom_feature_vector_to_dict, bond_feature_vector_to_dict)
from rdkit import Chem
import numpy as np


def ReorderCanonicalRankAtoms(mol):
    order = tuple(zip(*sorted([(j, i) for i, j in enumerate(Chem.CanonicalRankAtoms(mol))])))[1]
    mol_renum = Chem.RenumberAtoms(mol, order)
    return mol_renum, order

def smiles2graph(smiles_string, removeHs=True, reorder_atoms=False):
    """
    Converts SMILES string to graph Data object
    :input: SMILES string (str)
    :return: graph object
    """

    mol = Chem.MolFromSmiles(smiles_string)
    mol = mol if removeHs else Chem.AddHs(mol)
    if reorder_atoms:
        mol, _ = ReorderCanonicalRankAtoms(mol)

    # atoms
    atom_features_list = []
    for atom in mol.GetAtoms():
        atom_features_list.append(atom_to_feature_vector(atom))
    x = np.array(atom_features_list, dtype = np.int64)

    # bonds
    num_bond_features = 3  # bond type, bond stereo, is_conjugated
    if len(mol.GetBonds()) > 0: # mol has bonds
        edges_list = []
        edge_features_list = []
        for bond in mol.GetBonds():
            i = bond.GetBeginAtomIdx()
            j = bond.GetEndAtomIdx()

            edge_feature = bond_to_feature_vector(bond)

            # add edges in both directions
            edges_list.append((i, j))
            edge_features_list.append(edge_feature)
            edges_list.append((j, i))
            edge_features_list.append(edge_feature)

        # data.edge_index: Graph connectivity in COO format with shape [2, num_edges]
        edge_index = np.array(edges_list, dtype = np.int64).T

        # data.edge_attr: Edge feature matrix with shape [num_edges, num_edge_features]
        edge_attr = np.array(edge_features_list, dtype = np.int64)

    else:   # mol has no bonds
        edge_index = np.empty((2, 0), dtype = np.int64)
        edge_attr = np.empty((0, num_bond_features), dtype = np.int64)

    graph = dict()
    graph['edge_index'] = edge_index
    graph['edge_feat'] = edge_attr
    graph['node_feat'] = x
    graph['num_nodes'] = len(x)

    return graph


if __name__ == '__main__':
    #graph = smiles2graph('O1C=C[C@H]([C@H]1O2)c3c2cc(OC)c4c3OC(=O)C5=C4CCC(=O)5')
    #print(graph)
    #print(type(graph))
    targets = df['HIV_active']
    data_list = [smiles2graph(mol) for mol in df['smiles']]

[12:18:29] WARNING: not removing hydrogen atom without neighbors
[12:18:29] WARNING: not removing hydrogen atom without neighbors


In [ ]:
data_df = pd.DataFrame(data_list)
final_data = pd.concat([data_df,targets],axis=1)
print(final_data.head(5))
print(len(final_data))

                                          edge_index  \
0  [[0, 1, 1, 2, 2, 3, 3, 4, 4, 5, 5, 6, 6, 7, 7,...   
1  [[0, 1, 1, 2, 2, 3, 3, 4, 4, 5, 5, 6, 6, 7, 0,...   
2  [[0, 1, 1, 2, 1, 3, 3, 4, 4, 5, 5, 6, 6, 7, 7,...   
3  [[0, 1, 1, 2, 2, 3, 3, 4, 4, 5, 5, 6, 6, 7, 7,...   
4  [[0, 1, 1, 2, 1, 3, 1, 4, 4, 5, 5, 6, 6, 7, 6,...   

                                           edge_feat  \
0  [[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [...   
1  [[1, 0, 1], [1, 0, 1], [0, 0, 1], [0, 0, 1], [...   
2  [[0, 0, 0], [0, 0, 0], [1, 0, 1], [1, 0, 1], [...   
3  [[0, 0, 1], [0, 0, 1], [3, 0, 1], [3, 0, 1], [...   
4  [[1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [...   

                                           node_feat  num_nodes  HIV_active  
0  [[5, 0, 4, 5, 3, 0, 2, 0, 0], [5, 0, 4, 5, 2, ...         19           0  
1  [[5, 0, 3, 5, 1, 0, 1, 0, 0], [5, 0, 3, 5, 1, ...         39           0  
2  [[5, 0, 4, 5, 3, 0, 2, 0, 0], [5, 0, 3, 5, 0, ...         21           0  
3  [[6, 0, 3, 

In [ ]:
print(data_list[0])
print(len(data_list))
print(len(targets))

{'edge_index': array([[ 0,  1,  1,  2,  2,  3,  3,  4,  4,  5,  5,  6,  6,  7,  7,  8,
         6,  9,  4, 10, 10, 11, 11, 12, 12, 13, 11, 14, 14, 15, 15, 16,
        16, 17, 15, 18,  9,  2, 18,  4],
       [ 1,  0,  2,  1,  3,  2,  4,  3,  5,  4,  6,  5,  7,  6,  8,  7,
         9,  6, 10,  4, 11, 10, 12, 11, 13, 12, 14, 11, 15, 14, 16, 15,
        17, 16, 18, 15,  2,  9,  4, 18]]), 'edge_feat': array([[0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
      

# Create Dataset Class
We create a custom dataset class to ensure that our dataset is in the correct format for the g-mixup functions we implemented.

In [ ]:
import torch
from torch_geometric.data import Dataset, Data

class CustomDataset(Dataset):
    def __init__(self, dataframe, transform=None, pre_transform=None):
        super(CustomDataset, self).__init__(transform, pre_transform)

        # Store your custom dataframe
        self.dataframe = dataframe

        # Perform any necessary preprocessing steps on your dataframe here

    def len(self):
        return len(self.dataframe)

    def get(self, idx):
        # Retrieve the graph data for the given index from your dataframe
        # and convert it to the required format for torch_geometric.Data

        # Extract relevant information from the dataframe
        graph_data = self.dataframe.iloc[idx]
        # Process the graph data and create a torch_geometric.Data object
        x = torch.tensor(graph_data['node_feat'], dtype=torch.float)
        y = torch.tensor(graph_data['HIV_active'], dtype=torch.long)
        edge_index = torch.tensor(graph_data['edge_index'], dtype=torch.long).t().contiguous()
        edge_attr = torch.tensor(graph_data['edge_feat'], dtype=torch.float)
        num_nodes = torch.tensor(graph_data['num_nodes'], dtype=torch.float)

        data = Data(x=x, y=y, edge_index=edge_index, edge_attr=edge_attr,num_nodes=num_nodes)

        if self.transform is not None:
            data = self.transform(data)

        return data

In [ ]:
custom_dataset = CustomDataset(dataframe=final_data)
print(custom_dataset[2])
print(custom_dataset.len())
print(custom_dataset.get(1))

Data(x=[21, 9], edge_index=[48, 2], edge_attr=[48, 3], y=0, num_nodes=21.0)
41127
Data(x=[39, 9], edge_index=[88, 2], edge_attr=[88, 3], y=0, num_nodes=39.0)


# *G* - Mixup
We implemented g-mixup functions, so that they would work for our intended purpose and our dataset. Although the following code is inspired by the idea proposed in the *G* - Mixup paper (https://arxiv.org/abs/2202.07179), it is not the same code.
The code from the *G* - Mixup paper can be found in this GitHub repository: https://github.com/ahxt/g-mixup.

In [ ]:
#g-mixup
import torch
from typing import List, Tuple
from torch_geometric.data import Data
import random
from torch_geometric.utils import degree , dense_to_sparse
import logging
from torch_geometric.utils import to_dense_adj
from torch_geometric.datasets import TUDataset
import torch.nn.functional as F
import copy

logger = logging.getLogger()
logger.setLevel(logging.DEBUG)
formatter = logging.Formatter('%(asctime)s - %(levelname)s: - %(message)s', datefmt='%Y-%m-%d')

def prepare_dataset_onehot_y(dataset):

    y_set = set()
    for data in dataset:
        y_set.add(int(data.y))
    num_classes = len(y_set)

    for data in dataset:
        data.y = F.one_hot(data.y, num_classes=num_classes).to(torch.float)[0]
    return dataset

def stat_graph(graphs_list: List[Data]):
    num_total_nodes = []
    num_total_edges = []
    for graph in graphs_list:
        num_total_nodes.append(graph.num_nodes)
        num_total_edges.append(  graph.edge_index.shape[1] )
    avg_num_nodes = sum( num_total_nodes ) / len(graphs_list)
    avg_num_edges = sum( num_total_edges ) / len(graphs_list) / 2.0
    avg_density = avg_num_edges / (avg_num_nodes * avg_num_nodes)

    median_num_nodes = np.median( num_total_nodes )
    median_num_edges = np.median(num_total_edges)
    median_density = median_num_edges / (median_num_nodes * median_num_nodes)

    return avg_num_nodes, avg_num_edges, avg_density, median_num_nodes, median_num_edges, median_density

def split_class_graphs(dataset):

    y_list = []
    for data in dataset:
        y_list.append(tuple(data.y.tolist()))
        # print(y_list)
    num_classes = len(set(y_list))

    all_graphs_list = []
    for graph in dataset:
        adj = to_dense_adj(graph.edge_index)[0].numpy()
        all_graphs_list.append(adj)

    class_graphs = []
    for class_label in set(y_list):
        c_graph_list = [all_graphs_list[i] for i in range(len(y_list)) if y_list[i] == class_label]
        class_graphs.append( ( np.array(class_label), c_graph_list ) )

    return class_graphs

def align_graphs(graphs: List[np.ndarray],
                 padding: bool = False, N: int = None) -> Tuple[List[np.ndarray], List[np.ndarray], int, int]:
    """
    Align multiple graphs by sorting their nodes by descending node degrees
    :param graphs: a list of binary adjacency matrices
    :param padding: whether padding graphs to the same size or not
    :return:
        aligned_graphs: a list of aligned adjacency matrices
        normalized_node_degrees: a list of sorted normalized node degrees (as node distributions)
    """
    num_nodes = [graphs[i].shape[0] for i in range(len(graphs))]
    max_num = max(num_nodes)
    min_num = min(num_nodes)

    aligned_graphs = []
    normalized_node_degrees = []
    for i in range(len(graphs)):
        num_i = graphs[i].shape[0]

        node_degree = 0.5 * np.sum(graphs[i], axis=0) + 0.5 * np.sum(graphs[i], axis=1)
        node_degree /= np.sum(node_degree)
        idx = np.argsort(node_degree)  # ascending
        idx = idx[::-1]  # descending

        sorted_node_degree = node_degree[idx]
        sorted_node_degree = sorted_node_degree.reshape(-1, 1)

        sorted_graph = copy.deepcopy(graphs[i])
        sorted_graph = sorted_graph[idx, :]
        sorted_graph = sorted_graph[:, idx]

        max_num = max(max_num, N)

        if padding:
            # normalized_node_degree = np.ones((max_num, 1)) / max_num
            normalized_node_degree = np.zeros((max_num, 1))
            normalized_node_degree[:num_i, :] = sorted_node_degree

            aligned_graph = np.zeros((max_num, max_num))
            aligned_graph[:num_i, :num_i] = sorted_graph

            normalized_node_degrees.append(normalized_node_degree)
            aligned_graphs.append(aligned_graph)
        else:
            normalized_node_degrees.append(sorted_node_degree)
            aligned_graphs.append(sorted_graph)

        if N:
            aligned_graphs = [aligned_graph[:N, :N] for aligned_graph in aligned_graphs]
            normalized_node_degrees = normalized_node_degrees[:N]

    return aligned_graphs, normalized_node_degrees, max_num, min_num

def graph_numpy2tensor(graphs: List[np.ndarray]) -> torch.Tensor:
    """
    Convert a list of np arrays to a pytorch tensor
    :param graphs: [K (N, N) adjacency matrices]
    :return:
        graph_tensor: [K, N, N] tensor
    """
    graph_tensor = np.array(graphs)
    return torch.from_numpy(graph_tensor).float()

def universal_svd(aligned_graphs: List[np.ndarray], threshold: float = 2.02) -> np.ndarray:
    """
    Estimate a graphon by universal singular value thresholding.
    Reference:
    Chatterjee, Sourav.
    "Matrix estimation by universal singular value thresholding."
    The Annals of Statistics 43.1 (2015): 177-214.
    :param aligned_graphs: a list of (N, N) adjacency matrices
    :param threshold: the threshold for singular values
    :return: graphon: the estimated (r, r) graphon model
    """
    aligned_graphs = graph_numpy2tensor(aligned_graphs)
    num_graphs = aligned_graphs.size(0)

    if num_graphs > 1:
        sum_graph = torch.mean(aligned_graphs, dim=0)
    else:
        sum_graph = aligned_graphs[0, :, :]  # (N, N)

    num_nodes = sum_graph.size(0)

    u, s, v = torch.svd(sum_graph)
    singular_threshold = threshold * (num_nodes ** 0.5)
    binary_s = torch.lt(s, singular_threshold)
    s[binary_s] = 0
    graphon = u @ torch.diag(s) @ torch.t(v)
    graphon[graphon > 1] = 1
    graphon[graphon < 0] = 0
    graphon = graphon.numpy()
    return graphon

def two_graphons_mixup(two_graphons, la=0.5, num_sample=20):

    label = la * two_graphons[0][0] + (1 - la) * two_graphons[1][0]
    new_graphon = la * two_graphons[0][1] + (1 - la) * two_graphons[1][1]

    sample_graph_label = torch.from_numpy(label).type(torch.float32)
    # print(new_graphon)

    sample_graphs = []
    for i in range(num_sample):

        sample_graph = (np.random.rand(*new_graphon.shape) <= new_graphon).astype(np.int32)
        sample_graph = np.triu(sample_graph)
        sample_graph = sample_graph + sample_graph.T - np.diag(np.diag(sample_graph))

        sample_graph = sample_graph[sample_graph.sum(axis=1) != 0]
        sample_graph = sample_graph[:, sample_graph.sum(axis=0) != 0]

        A = torch.from_numpy(sample_graph)
        edge_index, _ = dense_to_sparse(A)

        num_nodes = int(torch.max(edge_index)) + 1

        pyg_graph = Data()
        pyg_graph.y = sample_graph_label
        pyg_graph.edge_index = edge_index
        pyg_graph.num_nodes = num_nodes
        sample_graphs.append(pyg_graph)

        # print(edge_index)
    return sample_graphs

def prepare_dataset_x(dataset):
    if dataset[0].x is None:
        max_degree = 0
        degs = []
        for data in dataset:
            degs += [degree(data.edge_index[0], dtype=torch.long)]
            max_degree = max( max_degree, degs[-1].max().item() )
            data.num_nodes = int( torch.max(data.edge_index) ) + 1

        if max_degree < 2000:
            # dataset.transform = T.OneHotDegree(max_degree)

            for data in dataset:
                degs = degree(data.edge_index[0], dtype=torch.long)
                data.x = F.one_hot(degs, num_classes=max_degree+1).to(torch.float)
        else:
            deg = torch.cat(degs, dim=0).to(torch.float)
            mean, std = deg.mean().item(), deg.std().item()
            for data in dataset:
                degs = degree(data.edge_index[0], dtype=torch.long)
                data.x = ( (degs - mean) / std ).view( -1, 1 )
    return dataset


seed = 1314

dataset = custom_dataset
dataset = list(dataset)

for graph in dataset:
    graph.y = graph.y.view(-1)

dataset = prepare_dataset_onehot_y(dataset)


random.seed(seed)
random.shuffle( dataset )

train_nums = int(len(dataset) * 0.7)
train_val_nums = int(len(dataset) * 0.8)

avg_num_nodes, avg_num_edges, avg_density, median_num_nodes, median_num_edges, median_density = stat_graph(dataset[: train_nums])
logger.info(f"avg num nodes of training graphs: { avg_num_nodes }")
logger.info(f"avg num edges of training graphs: { avg_num_edges }")
logger.info(f"avg density of training graphs: { avg_density }")
logger.info(f"median num nodes of training graphs: { median_num_nodes }")
logger.info(f"median num edges of training graphs: { median_num_edges }")
logger.info(f"median density of training graphs: { median_density }")

resolution = int(median_num_nodes)

class_graphs = split_class_graphs(dataset[:train_nums])
graphons = []
ge = 'USVT'
for label, graphs in class_graphs:

        logger.info(f"label: {label}, num_graphs:{len(graphs)}" )
        align_graphs_list, normalized_node_degrees, max_num, min_num = align_graphs(
            graphs, padding=True, N=resolution)
        logger.info(f"aligned graph {align_graphs_list[0].shape}" )

        logger.info(f"ge: {ge}")
        graphon = universal_svd(align_graphs_list, threshold=0.2)
        graphons.append((label, graphon))


for label, graphon in graphons:
    logger.info(f"graphon info: label:{label}; mean: {graphon.mean()}, shape, {graphon.shape}")

aug_ratio = 0.15
aug_num = 10
lam_range = [0.005, 0.01]


num_sample = int( train_nums * aug_ratio / aug_num )
lam_list = np.random.uniform(low=lam_range[0], high=lam_range[1], size=(aug_num,))

random.seed(seed)
new_graph = []
for lam in lam_list:
    logger.info( f"lam: {lam}" )
    logger.info(f"num_sample: {num_sample}")
    two_graphons = random.sample(graphons, 2)
    new_graph += two_graphons_mixup(two_graphons, la=lam, num_sample=num_sample)
    logger.info(f"label: {new_graph[-1].y}")

avg_num_nodes, avg_num_edges, avg_density, median_num_nodes, median_num_edges, median_density = stat_graph(new_graph)
logger.info(f"avg num nodes of new graphs: { avg_num_nodes }")
logger.info(f"avg num edges of new graphs: { avg_num_edges }")
logger.info(f"avg density of new graphs: { avg_density }")
logger.info(f"median num nodes of new graphs: { median_num_nodes }")
logger.info(f"median num edges of new graphs: { median_num_edges }")
logger.info(f"median density of new graphs: { median_density }")

dataset = new_graph + dataset
logger.info( f"real aug ratio: {len( new_graph ) / train_nums }" )
train_nums = train_nums + len( new_graph )
train_val_nums = train_val_nums + len( new_graph )

dataset = prepare_dataset_x( dataset )

logger.info(f"num_features: {dataset[0].x.shape}" )
logger.info(f"num_classes: {dataset[0].y.shape}"  )

In [ ]:
print(len(dataset))

45437
